In [ ]:
import sys
import datetime

import numpy as np
import pandas as pd
import pytz as pytz
from matplotlib import pyplot as plt
from sklearn.metrics import r2_score, mean_absolute_percentage_error

from cnn.helpers import train
from cnn.helpers import predict
from cnn.preprocessors import process_inputs, process_targets

In [ ]:
# Đường dẫn đến thư mục training_set
training_set_path = '/content/drive/MyDrive/stock_analysis/training_set/'

# Đọc dữ liệu từ tệp CSV
data = pd.read_csv(training_set_path + 'VIC.csv')

# Tạo cột 'Date' từ các cột 'year', 'month', 'day_of_month'
data['Date'] = pd.to_datetime(data['year'].astype(str) + '-' +
                                  data['month'].astype(str) + '-' +
                                  data['day_of_month'].astype(str),
                                  errors='coerce')


# Thiết lập cột 'Date' làm chỉ số
data.set_index('Date', inplace=True)

# Sử dụng data 'close_price'
price_series = data['close_price'].dropna()

# Chuyển đổi cột 'close_price' về kiểu số
price_series = pd.to_numeric(price_series, errors='coerce')

# Kiểm tra kiểu dữ liệu trong price_series
print(price_series.apply(type).value_counts())

# Loại bỏ các giá trị không hợp lệ sau khi chuyển đổi
price_series = price_series.dropna()

print(price_series)


In [ ]:
x_df = process_inputs(price_series, window_length=10).dropna()
y_series = process_targets(price_series).dropna()

#Only keep rows có cả data in and out
common_index = x_df.index.intersection(y_series.index)
x_df, y_series = x_df.loc[common_index], y_series.loc[common_index]

In [ ]:
import datetime
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score, mean_absolute_percentage_error

# Giả sử x_df là DataFrame chứa dữ liệu đầu vào và y_series là Series chứa giá trị close_price

r2_list = []  # Lưu trữ R Squared của từng lần kiểm tra
mape_list = []  # Lưu trữ MAPE của từng lần kiểm tra
corr_list = []  # Lưu trữ các hệ số tương quan của từng lần kiểm tra
forecasts = []  # Lưu trữ dự đoán của từng lần kiểm tra

training_start = datetime.datetime(2015, 1, 1)  # Khởi đầu khoảng thời gian huấn luyện

# Walk-Forward Validation với Cửa sổ Mở Rộng
for training_year in range(2016, datetime.date.today().year):

    training_cutoff = datetime.datetime(training_year, 1, 1)
    test_cutoff = datetime.datetime(training_year + 1, 1, 1)

    # Chia dữ liệu huấn luyện (cửa sổ mở rộng)
    training_x_series = x_df.loc[(x_df.index >= training_start) & (x_df.index < training_cutoff)]
    training_y_series = y_series.loc[(y_series.index >= training_start) & (y_series.index < training_cutoff)]

    # Chia dữ liệu kiểm tra
    test_x_series = x_df.loc[(x_df.index >= training_cutoff) & (x_df.index < test_cutoff)]
    test_y_series = y_series.loc[(y_series.index >= training_cutoff) & (y_series.index < test_cutoff)]

    # Huấn luyện mô hình
    trained_model = train(training_x_series, training_y_series, epochs=100)

    # Dự đoán trên tập kiểm tra
    forecast_series = predict(trained_model, test_x_series)
    results_df = forecast_series.to_frame('Forecast').join(test_y_series.to_frame('Actual')).dropna()
    forecasts.append(results_df)

    # Đánh giá dự đoán
    r2 = r2_score(results_df['Actual'], results_df['Forecast'])
    mape = mean_absolute_percentage_error(results_df['Actual'], results_df['Forecast'])
    corr = results_df['Actual'].corr(results_df['Forecast'])

    r2_list.append(r2)
    mape_list.append(mape)
    corr_list.append(corr)

    print(f"{training_year} R Squared: {r2:.4f}, MAPE: {mape:.4f}, Correlation: {corr:.4f}")

# Tổng hợp kết quả
print(f"\nAverage R Squared: {np.mean(r2_list):.4f}")
print(f"Average MAPE: {np.mean(mape_list):.4f}")
print(f"Average Correlation: {np.mean(corr_list):.4f}")
